<a href="https://colab.research.google.com/github/paveethranswam/Speech-style-transfer-and-ASR/blob/main/Wav2Vec2_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers  datasets librosa jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 10.3 MB/s 
     |████████████████████████████████| 451 kB 4.4 MB/s 
     |████████████████████████████████| 7.6 MB 27.8 MB/s 
     |████████████████████████████████| 182 kB 68.0 MB/s 
     |████████████████████████████████| 132 kB 39.7 MB/s 
     |████████████████████████████████| 212 kB 16.3 MB/s 
     |████████████████████████████████| 127 kB 63.3 MB/s 
     |████████████████████████████████| 1.4 MB 48.3 MB/s 
     |████████████████████████████████| 2.2 MB 19.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
!apt-get install libsox-fmt-all libsox-dev sox > /dev/null
!python -m pip install torchaudio > /dev/null
!python -m pip install git+https://github.com/facebookresearch/WavAugment.git > /dev/null
!git clone --recursive https://github.com/parlance/ctcdecode.git
!cd ctcdecode && pip install .

  Running command git clone -q https://github.com/facebookresearch/WavAugment.git /tmp/pip-req-build-62z8hho0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from datasets import load_dataset, load_metric,Audio
import transformers
import random
import numpy as np
import pandas as pd
import re
from IPython.display import display, HTML
from transformers import AutoProcessor
import librosa
import torch
from datasets import ClassLabel
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2ForCTC
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR
from transformers import get_scheduler
import augment
import json
from jiwer import wer
import os
import tqdm
# import ctcdecode
from functools import partial
from IPython.display import display, HTML

In [5]:
# Get device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda:0 device


In [6]:
minds = load_dataset("PolyAI/minds14", name="en-US", split="train[:100]")

Generating train split: 0 examples [00:00, ? examples/s]

Dataset minds14 downloaded and prepared to /root/.cache/huggingface/datasets/PolyAI___minds14/en-US/1.0.0/aa40414f15e0f919231d617440192034af844835dc1e6a697f4b552e0551fd26. Subsequent calls will reuse this data.


In [7]:
minds = minds.remove_columns(["english_transcription", "intent_class", "lang_id"])

In [8]:
minds

Dataset({
    features: ['path', 'audio', 'transcription'],
    num_rows: 100
})

In [9]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(minds.remove_columns(["path", "audio"]))

,transcription
0,what's my account balance
1,show me my account balance please
2,show me my account balance
3,I would like to make a down payment
4,hello how would I go about setting up a joint account with my partner
5,how can my wife and I set up a joint account
6,I'd like to make a payment on my credit card account
7,what is my account balance
8,set up a joint account
9,say hello I have to make a payment on my bill can you tell me how much I owe and


In [11]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).upper()
    return batch

minds = minds.map(remove_special_characters)

  0%|          | 0/100 [00:00<?, ?ex/s]

In [12]:
vocab_dict = {'vocab': [],'all_text': []}
def extract_all_chars(batch):
  global vocab_dict
  all_text = " ".join(batch["transcription"])
  vocab_list = list(set(all_text))
  extras = ["", "", "", ""]
  vocab = {v: k for k, v in enumerate(extras)}
  for k, v in enumerate(vocab_list):
    if v == " ":
      vocab["|"] = k+len(extras)
    else:
      vocab[v] = k+len(extras)
  vocab_dict['vocab'].extend(vocab.keys())
  vocab_dict['all_text'].append(all_text)
  return vocab_dict

m1 = minds.remove_columns(["path", "audio"])
vocabs = m1.map(extract_all_chars)

  0%|          | 0/100 [00:00<?, ?ex/s]

In [13]:
vocab_dict['vocab'] = list(set(vocab_dict['vocab']))

In [14]:
print(vocab_dict)

{'vocab': ['', 'H', 'F', 'E', 'N', 'U', 'B', 'W', '|', 'O', 'I', 'S', 'Y', 'D', 'C', 'L', 'A', 'T', 'P', 'R', 'M', 'K', "'", 'J', 'V', 'G'], 'all_text': ['I   W O U L D   L I K E   T O   S E T   U P   A   J O I N T   A C C O U N T   W I T H   M Y   P A R T N E R', 'H E N R Y   C O U N T Y   S E T   U P   A   J O I N T   A C C O U N T   W I T H   M Y   W I F E   A N D   W H E R E   A R E   T H E Y   A T', "H I   I ' D   L I K E   T O   S E T   U P   A   J O I N T   A C C O U N T   W I T H   M Y   P A R T N E R   I ' M   N O T   S E E I N G   T H E   O P T I O N   T O   D O   I T   O N   T H E   A P P   S O   I   C A L L E D   I N   T O   G E T   S O M E   H E L P   C A N   I   D O   I T   O V E R   T H E   P H O N E   W I T H   Y O U   A N D   G I V E   Y O U   T H E   I N F O R M A T I O N", 'H O W   D O   I   S T A R T   A   J O I N T   A C C O U N T', 'C A N   Y O U   H E L P   M E   S E T   U P   A   J O I N T   B A N K   A C C O U N T', 'H O W   T O   S E T   U P   A   J O I N T   

In [15]:
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base")

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [16]:
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))

In [17]:
minds = minds.train_test_split(test_size=0.2)

In [18]:
minds['train']['audio'][0]['array']

array([ 2.5668835e-06,  1.3541977e-04,  2.4167701e-04, ...,
       -2.0254294e-05,  2.1638248e-06,  1.5568066e-05], dtype=float32)

In [19]:
class SpeechTransform:
  def __init__(self, p=0.5):
    self.p = p

  def __call__(self, y):
    if np.random.rand() < self.p:
      return self.apply(y)
    return y

  def apply(self, y: np.ndarray):
      raise NotImplementedError

In [20]:
class ClipEffect(SpeechTransform):
  def __init__(self, p=0.5, sr=16000, rate=0.25):
    super().__init__(p)
    self.rate = rate
    self.sr = sr
  
  def apply(self, x):
    clip_chain = augment.EffectChain().clip(self.rate)
    return clip_chain.apply(x, src_info={'rate': self.sr})

In [21]:
class PitchShift(SpeechTransform):
  def __init__(self, p=0.5, sr=16000, max_rate=200):
    super().__init__(p)
    self.sr = sr
    self.max_rate = max_rate

  def apply(self, x):
    rate = np.random.randint(-self.max_rate, self.max_rate)
    return augment.EffectChain().pitch(rate).rate(self.sr).apply(x, src_info={'rate': self.sr})

In [22]:
class ReverbEffect(SpeechTransform):
  def __init__(self, p=0.5, sr=16000, max_rate=101):
    super().__init__(p)
    self.sr = sr
    self.max_rate = max_rate
  
  def apply(self, x):
    rnd_rate = np.random.randint(0, self.max_rate)
    return augment.EffectChain().reverb(50, 50, rnd_rate).channels(1).apply(x, src_info={'rate': self.sr})

In [23]:
class TimeDropout(SpeechTransform):
  def __init__(self, p=0.5, sr=16000, max_seconds=0.5):
    super().__init__(p)
    self.sr = sr
    self.max_seconds = max_seconds
  
  def apply(self, x):
    return augment.EffectChain().time_dropout(max_seconds=self.max_seconds).apply(x, src_info={'rate': self.sr})

In [24]:
class TimeShift(SpeechTransform):
  def __init__(self, p=0.5, sr=16000, min_shift=0.7, max_shift=2):
    super().__init__(p)
    self.sr = sr
    self.max_shift = max_shift
    self.min_shift = min_shift
  
  def apply(self, x):
    x = x.numpy()
    rate = np.random.uniform(self.min_shift, self.max_shift)
    return torch.as_tensor(librosa.effects.time_stretch(x, rate))

In [25]:
class Compose:
  def __init__(self, transforms):
    self.transforms = transforms
  def __call__(self, x):
    for tr in self.transforms:
      y = tr(x)
    return y

In [26]:
aug_transforms = Compose([ClipEffect(p=0.6),
                          PitchShift(p=0.6),
                          ReverbEffect(p=0.6),
                          TimeDropout(p=0.6),
                          TimeShift(p=0.8)])

In [27]:
minds

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription'],
        num_rows: 80
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription'],
        num_rows: 20
    })
})

In [28]:
def apply_transforms(batch):
      audio = batch["audio"] 
      sp_arr = aug_transforms(torch.Tensor(audio['array']))
      batch['audio']['array'] = sp_arr
      return batch

minds = minds.map(apply_transforms)

  0%|          | 0/80 [00:00<?, ?ex/s]

  0%|          | 0/20 [00:00<?, ?ex/s]

In [29]:
vocab_list = vocab_dict['vocab']

vocab_list = {v: k for k, v in enumerate(vocab_list)}
vocab_list

vocab_list["|"] = len(vocab_list)
del vocab_list['']
for key in vocab_list.keys():
  vocab_list[key] -= 1 
print(vocab_list)

{'H': 0, 'F': 1, 'E': 2, 'N': 3, 'U': 4, 'B': 5, 'W': 6, '|': 25, 'O': 8, 'I': 9, 'S': 10, 'Y': 11, 'D': 12, 'C': 13, 'L': 14, 'A': 15, 'T': 16, 'P': 17, 'R': 18, 'M': 19, 'K': 20, "'": 21, 'J': 22, 'V': 23, 'G': 24}


In [30]:
label_dict = {"": 0, 
              "": 1, 
              "": 2,
              "": 3,
              "|": 4,
              "E": 5,
              "T": 6,
              "A": 7,
              "O": 8,
              "N": 9,
              "I": 10,
              "H": 11,
              "S": 12,
              "R": 13,
              "D": 14,
              "L": 15,
              "U": 16,
              "M": 17,
              "W": 18,
              "C": 19,
              "F": 20,
              "G": 21,
              "Y": 22,
              "P": 23,
              "B": 24,
              "V": 25,
              "K": 26,
              "'": 27,
              "X": 28,
              "J": 29,
              "Q": 30,
              "Z": 31
              }

label_dict['[UNK]'] = len(label_dict)
label_dict['[PAD]'] = len(label_dict)
print(len(label_dict))
labels = [key for key, value in label_dict.items()]

31


In [31]:
vocab_list['[UNK]'] = len(vocab_list)
vocab_list['[PAD]'] = len(vocab_list)
print(len(vocab_list))

27


In [32]:
with open('/content/drive/MyDrive/vocab.json', 'w') as vocab_file:
    json.dump(label_dict, vocab_file)

In [33]:
# tokenizer = Wav2Vec2CTCTokenizer("/content/drive/MyDrive/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

In [34]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [35]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [36]:
minds["train"]

Dataset({
    features: ['path', 'audio', 'transcription'],
    num_rows: 80
})

In [37]:
rand_int = random.randint(0, len(minds["train"]))

print("Target text:", minds["train"][rand_int]["transcription"])
print("Input array shape:", np.asarray(minds["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", minds["train"][rand_int]["audio"]["sampling_rate"])

Target text: SHOW ME MY ACCOUNT BALANCE
Input array shape: (66914,)
Sampling rate: 16000


In [38]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

In [39]:
minds_tr = minds.map(prepare_dataset, remove_columns=minds.column_names["train"], num_proc=4)

#0:   0%|          | 0/20 [00:00<?, ?ex/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


#1:   0%|          | 0/20 [00:00<?, ?ex/s]

#2:   0%|          | 0/20 [00:00<?, ?ex/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


...

#1:   0%|          | 0/5 [00:00<?, ?ex/s]

#2:   0%|          | 0/5 [00:00<?, ?ex/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


#3:   0%|          | 0/5 [00:00<?, ?ex/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [40]:
def collate_function(batch, processor, padding=True, max_length=None, 
                     max_length_labels=None, pad_to_multiple_of=None, pad_to_multiple_of_labels=None):
    b_X = [{"input_values": sample["input_values"]} for sample in batch]
    b_Y = [{"input_ids": sample["labels"]} for sample in batch]
    features = processor.pad(b_X,
              padding=padding,
              max_length=max_length,
              pad_to_multiple_of=pad_to_multiple_of,
              return_tensors="pt")

    with processor.as_target_processor():
        batchY = processor.pad(b_Y,
          padding=padding,
          max_length=max_length_labels,
          pad_to_multiple_of=pad_to_multiple_of_labels,
          return_tensors="pt")
    labels = batchY["input_ids"].masked_fill(batchY.attention_mask.ne(1), -100)
    features["labels"] = labels
    return features

In [41]:
collate_fn = partial(collate_function, processor=processor, padding=True, max_length=None, 
                     max_length_labels=None, pad_to_multiple_of=None, pad_to_multiple_of_labels=None)

In [42]:
version = 2
model_save_path = "/content/drive/MyDrive/wav2vec_finetuning/"
model_name = f"wav2vec_{version}"
batch_size =3
epochs = 100
lr = 0.0002

In [43]:
train_dataloader = torch.utils.data.DataLoader(minds_tr["train"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [44]:
batch_sample = next(iter(train_dataloader))

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [45]:
print(batch_sample.keys())

dict_keys(['input_values', 'labels'])


In [46]:
print(batch_sample["input_values"].shape)
print(batch_sample["labels"].shape)

torch.Size([3, 178751])
torch.Size([3, 128])


In [47]:
batch_sample['labels'][0]

tensor([  11,    8,   18,    4,   17,   16,   19,   11,    4,   10,    9,   20,
           8,   13,   17,    7,    6,   10,    8,    9,    4,    7,   24,    8,
          16,    6,    4,   12,   10,   21,    9,   10,    9,   21,    4,   16,
          23,    4,   20,    8,   13,    4,    7,    4,   29,    8,   10,    9,
           6,    4,    7,   19,   19,    8,   16,    9,    6, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100])

In [48]:
print(len(train_dataloader))

27


In [49]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-960h-lv60-self",
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    ignore_mismatched_sizes = True)

Downloading:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
os.makedirs(model_save_path, exist_ok=True)

In [51]:
model.freeze_feature_encoder()

In [52]:
def compute_metrics(labels, preds):
    preds = torch.argmax(preds, axis=-1)
    labels[labels == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.batch_decode(preds)
    label_str = processor.batch_decode(labels, group_tokens=False)
    return wer(label_str, pred_str)

In [53]:
optimizer = AdamW(model.parameters(), lr=lr)

In [54]:
num_training_steps = epochs*len(train_dataloader)

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [55]:
scaler = torch.cuda.amp.GradScaler()

In [56]:
model = model.to(device)
model = model.train()

In [57]:
torch.save(model, os.path.join(model_save_path, model_name+".pt"))
processor.save_pretrained(os.path.join(model_save_path, model_name+"_vocab"))

In [58]:
def train_step(train_dataloader, optimizer, lr_scheduler, processor, verbose_at=5, print_sentences=False):
    losses = []
    wers = []
    for step, data in enumerate(train_dataloader):
        optimizer.zero_grad()
        # move inputs to device
        batch = {k: v.to(device) for k, v in data.items()}
        with torch.cuda.amp.autocast():
          outputs = model(**batch)
        loss = outputs.loss
        losses.append(loss.cpu().detach().numpy())
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        lr_scheduler.step()
        preds = outputs.logits
        out_proba = torch.nn.functional.softmax(preds, dim=-1)
        labels = data["labels"]
        metrics = compute_metrics(labels, out_proba)
        wers.append(metrics)
        if step%verbose_at == 0:
            print(f"WER: {metrics} \t LOSS: {loss.cpu().detach().numpy()}")
            if print_sentences:
              for l, p in zip(labels, preds):
                  predicted_str = processor.tokenizer.decode(torch.argmax(p, dim =-1))
                  label_str = processor.tokenizer.decode(l)
                  print(f"True: {label_str}; Predicted: {predicted_str}")
    return {"loss": np.mean(losses), "wer":np.mean(wers)}

In [59]:
no_epochs = range(epochs)
epochs_progress_bar = tqdm.tqdm(no_epochs)
curr_best_loss = 1e10
for n in no_epochs:
    res = train_step(train_dataloader, optimizer, lr_scheduler, 
                     processor, verbose_at=len(train_dataloader)//2)
    res["best_loss"] = curr_best_loss
    if curr_best_loss > res["loss"]:
        print("Best model, saving at ",model_save_path)
        torch.save(model, os.path.join(model_save_path, model_name+".pt"))
        curr_best_loss = res["loss"]
        res["best_loss"] = curr_best_loss
    epochs_progress_bar.update(1)

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


WER: 0.9423076923076923 	 LOSS: 2.7560536861419678
WER: 1.0 	 LOSS: 2.9685606956481934
WER: 1.0 	 LOSS: 4.121778964996338
Best model, saving at  /content/drive/MyDrive/wav2vec_finetuning/


  1%|          | 1/100 [00:20<33:41, 20.41s/it]

WER: 1.0 	 LOSS: 3.584446668624878
WER: 1.0 	 LOSS: 3.261974334716797
WER: 1.0 	 LOSS: 3.0284783840179443
Best model, saving at  /content/drive/MyDrive/wav2vec_finetuning/


  2%|▏         | 2/100 [00:33<26:09, 16.02s/it]

...

WER: 1.0 	 LOSS: 2.9812986850738525
WER: 1.0 	 LOSS: 3.2118372917175293
WER: 1.0 	 LOSS: 3.140963315963745
WER: 1.0 	 LOSS: 3.133296489715576
Best model, saving at  /content/drive/MyDrive/wav2vec_finetuning/


 99%|█████████▉| 99/100 [16:36<00:09,  9.84s/it]

WER: 1.0 	 LOSS: 3.0052642822265625
WER: 1.0 	 LOSS: 2.8773484230041504


100%|██████████| 100/100 [16:46<00:00,  9.86s/it]

WER: 1.0 	 LOSS: 3.060624599456787


In [60]:
model_path = os.path.join(model_save_path, model_name+".pt")
pipeline_path= os.path.join(model_save_path, model_name+"_vocab")

In [61]:
model = model.eval()

In [ ]:
minds_te = minds.map(apply_transforms, remove_columns=minds.column_names["test"])
minds_te = minds_te.map(prepare_dataset, remove_columns=minds_te.column_names["test"], num_proc=4)

In [ ]:
print(minds_te)

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 80
    })
    test: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 20
    })
})


In [ ]:
class CTCBeamDecoder:
    def __init__(self, labels, blank_id=0, beam_size=100, kenlm_path=None):
        print("loading beam search with kenlm...")
        self.labels = labels

        # model_path = is the path to your external kenlm language model(LM). Default is none.
        # alpha = Weighting associated with the LMs probabilities. A weight of 0 means the LM has no effect.
        # beta = Weight associated with the number of words within our beam.

        self.ctcdecoder = ctcdecode.CTCBeamDecoder(
            self.labels, model_path=kenlm_path,
            alpha=0.6, beta=1,
            beam_width=beam_size, blank_id=blank_id)
        print("loading finished")

    def __call__(self, output, num_sentences=1):
        sentences = []
        for num in range(num_sentences):
          beam_result, beam_scores, timesteps, out_seq_len = self.ctcdecoder.decode(output)
          # beam_result[0][0][:out_seq_len[0][0]] get the top beam for the first item in batch
          sentences.append(self.output(beam_result[0][num], self.labels, out_seq_len[0][num]))
        return sentences

    def output(self, tokens, vocab, seq_len):
        out = ''.join([vocab[x] for x in tokens[0:seq_len]])
        # wave2vec implementation use | for space in vocabulary
        return out.replace("|", " ")

In [ ]:
decode_and_rescore = CTCBeamDecoder(kenlm_path=None, 
                                          labels=labels, 
                                          blank_id=label_dict.get(""), 
                                          beam_size=100)

In [ ]:
test_dataloader = torch.utils.data.DataLoader(minds_te["test"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [ ]:
for step, data in enumerate(test_dataloader):
            batch = {k: v for k, v in data.items()}
            with torch.cuda.amp.autocast():
              outputs = model(**batch)
              logits = outputs.logits
              out_proba = torch.nn.functional.softmax(logits, dim=-1)
              transcriptions = decode_and_rescore(out_proba, num_sentences=1)
              print(transcriptions)
              print(tokenizer.decode(data['labels'][0]))

s k a k a a a a k a a a a a a a a a a a a s s g e a s s s e a g e a a a g a a g a g a c g c k c c s c s f o s o e s e s s s s h h h e o e s s e s e u r e h o u o u a u u a a c c c c m s e e m m s e e v s s d s s s s k k o s s e s s s s s s s b s s s s s s s f f ' ' f f f f f f a a s h s v s s e k ' e e e e e e e e e ' h h s o b h h f f e d s s s s s s s s s e e m m m e e f u u s s s k s s h d d s s s s e e e e e b b b b b s c s s y e e e s s s e f a a     c j j j d e e e e e e e e e u j j j d c c c d s s e e e s s s s s s k k r c j j f r r s r a a a a a a g a g a a g a s g s g a s a s s s s s s s [PAD] s g s s s s g s s a g a s e a s s s e s e a s s a e s s s a e s s s a g s s s a s a s s s s a s e s s [PAD] s s s s [PAD] s g a g a g g s s g s g a g c k k k k k k k k c k   k c c c c k d e e k [UNK] s o o   k c k k s s s s s k k k c c c g a g g g a a g a g g k g c e e g g a a a a k a s g s s a s s s s s s g s s s s a s s s s s s e s e s s s a s s s s a s a s g s s a s e a s a s e a s s 

KeyboardInterrupt: ignored

In [62]:
# next(iter(test_dataloader))['input_values']